## TRABALHO DE CONCLUSÃO DE CURSO EM CIÊNCIA DE DADOS - PUC MINAS
## Aluno: Anderson Lhoret
## MODELO PREDITIVO DE DURAÇÃO DE OPERAÇÃO DE TRÂNSITO ADUANEIRO
# Notebook 2: Consolidação das bases e Tratamento dos Dados

In [27]:
import datetime
import numpy as np
import pandas as pd
import re

## 1. Inicialização dos datasets

In [28]:
# Base do Transito Aduaneiro
transito_csv = 'transito_aduaneiro.csv'

# Base do Google Maps - API Elevation
google_elevacao_csv = 'tabela_coordenadas_elevacao.csv'

# Base do Google Maps - API Distance Matrix
google_distancias_csv = 'tabela_distancias_ok.csv'

# Dataset gerado com a consolidação das bases
output_file = 'dataset_tratado.pkl'

In [29]:
# Leitura dos dados em arquivo CSV e carregamento em DataFrame
df = pd.read_csv(transito_csv)
df_e = pd.read_csv(google_elevacao_csv)
df_d = pd.read_csv(google_distancias_csv)

In [30]:
# Imprime as colunas de cada arquivo
print('transito_csv: ' + str(list(df)))
print()
print('google_elevacao_csv: ' + str(list(df_e)))
print()
print('google_distancias_csv: ' + str(list(df_d)))

transito_csv: ['dataHoraInicioTransito', 'dataHoraChegadaTransito', 'peso_bruto', 'origem_codUrf', 'origem_codigoRecinto', 'origem_latitude', 'origem_longitude', 'dest_realizado_codUrf', 'dest_realizado_codigoRecinto', 'dest_realizado_latitude', 'dest_realizado_longitude']

google_elevacao_csv: ['latitude', 'longitude', 'elevacao']

google_distancias_csv: ['origem_latitude', 'origem_longitude', 'dest_realizado_latitude', 'dest_realizado_longitude', 'distancia_text', 'distancia_value', 'duracao_text_Google', 'duracao_value_Google', 'endereco_origem', 'endereco_destino']


## 2. Tratamento da base de dados do Trânsito Aduaneiro

In [31]:
# Cria um campo com a concatenação dos códigos de Unidade Aduaneira da origem e do destino para representar a viagem
df['orig&dest'] = df['origem_codUrf'].astype(str) + " " + df['dest_realizado_codUrf'].astype(str)

In [32]:
# Remove as colunas relativas à Recinto e Unidade Aduaneira que não serão mais utilizadas
df.drop(columns=['origem_codUrf', 'origem_codigoRecinto', 'dest_realizado_codUrf', 'dest_realizado_codigoRecinto'], inplace=True)

In [33]:
df.head()

,dataHoraInicioTransito,dataHoraChegadaTransito,peso_bruto,origem_latitude,origem_longitude,dest_realizado_latitude,dest_realizado_longitude,orig&dest
0,2018-11-06 10:30:16,2018-11-09 12:01:08,5064.00,-29.894283,-51.173923,-23.928333,-46.369167,1017800 817800
1,2019-01-15 15:56:09,2019-01-16 16:53:34,14006.72,-22.313276,-49.133789,-25.511146,-54.565616,810300 917500
2,2019-01-15 15:06:30,2019-01-16 18:49:45,10005.00,-23.008442,-47.144659,-23.425529,-46.487609,817700 817600
3,2019-01-15 15:52:30,2019-01-16 16:53:01,15605.88,-22.313276,-49.133789,-25.511146,-54.565616,810300 917500
4,2019-01-15 15:01:18,2019-01-17 12:13:32,13588.45,-29.894283,-51.173923,-23.425529,-46.487609,1017800 817600


In [34]:
# Converte data e hora de início e chegada para tipo 'datetime64' e salva nas colunas 'dt_inicio' e 'dt_conclusao'
df.insert(0, 'dt_inicio', pd.to_datetime(df['dataHoraInicioTransito'], format='%Y-%m-%d %H:%M:%S'))
df.insert(1, 'dt_conclusao', pd.to_datetime(df['dataHoraChegadaTransito'], format='%Y-%m-%d %H:%M:%S'))

In [35]:
# Adiciona variável categórica da Hora Do Dia (HDD)
# Esta variável representa a hora em que teve início a operação de trânsito
df['HDD'] = pd.cut(df.dt_inicio.dt.hour, 
                    bins = list(range(0, 25, 1)),
                    right = False,
                    include_lowest = True,
                    labels = list(range(0, 24, 1)))

In [36]:
# Define uma nova coluna para a variável categórica de Dia Da Semana (DDS)
# Esta variável representa o dia da semana em que teve início a operação de trânsito
df['DDS'] = df['dt_inicio'].dt.day_name()

In [37]:
# Criação das variáveis dummy para a categoria Dia Da Semana
# df_dummies = pd.get_dummies(df['DDS'])
# df = pd.concat([df, df_dummies], axis=1)
# df.drop(['DDS'], inplace=True, axis=1)

In [38]:
# Calcula o tempo de duração do trânsito, em horas e salva na coluna 'duracao'
df['duracao'] = (df['dt_conclusao'] - df['dt_inicio']) / pd.Timedelta(hours=1)

In [39]:
df.head()

,dt_inicio,dt_conclusao,dataHoraInicioTransito,dataHoraChegadaTransito,peso_bruto,origem_latitude,origem_longitude,dest_realizado_latitude,dest_realizado_longitude,orig&dest,HDD,DDS,duracao
0,2018-11-06 10:30:16,2018-11-09 12:01:08,2018-11-06 10:30:16,2018-11-09 12:01:08,5064.00,-29.894283,-51.173923,-23.928333,-46.369167,1017800 817800,10,Tuesday,73.514444
1,2019-01-15 15:56:09,2019-01-16 16:53:34,2019-01-15 15:56:09,2019-01-16 16:53:34,14006.72,-22.313276,-49.133789,-25.511146,-54.565616,810300 917500,15,Tuesday,24.956944
2,2019-01-15 15:06:30,2019-01-16 18:49:45,2019-01-15 15:06:30,2019-01-16 18:49:45,10005.00,-23.008442,-47.144659,-23.425529,-46.487609,817700 817600,15,Tuesday,27.720833
3,2019-01-15 15:52:30,2019-01-16 16:53:01,2019-01-15 15:52:30,2019-01-16 16:53:01,15605.88,-22.313276,-49.133789,-25.511146,-54.565616,810300 917500,15,Tuesday,25.008611
4,2019-01-15 15:01:18,2019-01-17 12:13:32,2019-01-15 15:01:18,2019-01-17 12:13:32,13588.45,-29.894283,-51.173923,-23.425529,-46.487609,1017800 817600,15,Tuesday,45.203889


In [40]:
# Exclui as colunas originais de data/hora
df.drop(columns=['dataHoraInicioTransito','dataHoraChegadaTransito'], inplace=True)

## 3. Base do Google Maps - API Elevation

Incorpora ao DataFrame principal e trata as informações da base oriunda da API Elevation do Google Maps:
 - Elevação da origem
 - Elevação do destino


In [41]:
# Merge da variavel elevação para o local de origem
df = df.merge(df_e,
              left_on=['origem_latitude','origem_longitude'],
              right_on=['latitude','longitude'],
              how='left')

# Elimina as colunas 'latitude' e 'longitude' do dataframe principal
df.drop(columns=['latitude','longitude'], inplace=True)

# Renomeia a coluna recem incorporada 'elevacao' para identificar a elevação do ponto de origem
df.rename(columns={"elevacao": "elevacao_origem"}, inplace=True)

# Merge da variavel elevação para o local de destino
df = df.merge(df_e,
              left_on=['dest_realizado_latitude','dest_realizado_longitude'],
              right_on=['latitude','longitude'],
              how='left')

# Elimina as colunas 'latitude' e 'longitude' do dataframe principal
df.drop(columns=['latitude','longitude'], inplace=True)

# Renomeia a coluna recem incorporada 'elevacao' para identificar a elevação do ponto de destino
df.rename(columns={"elevacao": "elevacao_destino"}, inplace=True)

In [42]:
# Cria uma nova coluna 'delta_elevacao' representando a diferença na altitude entre ponto de destino e origem
df['delta_elevacao'] = df['elevacao_destino'] - df['elevacao_origem']

## 4. Base do Google Maps - API Distance Matrix

Incorpora ao DataFrame principal e trata as informações da base oriunda da API Distance Matrix do Google Maps:
 - Distância entre origem e destino
 - Tempo de viagem rodoviária entre origem e destino

In [43]:
# Converte a distancia 'x,xxx Km' para valor numérico 'xxxx'
df_d['distancia'] = df_d['distancia_text'].str.replace(' km', '')
df_d['distancia'] = df_d['distancia'].str.replace(',', '')
df_d['distancia'] = df_d['distancia'].astype(float)
# df_d.drop(columns=['distancia_text','distancia_value'])

In [44]:
# Merge da variavel distancia para o par origem-destino
df = df.merge(df_d[['origem_latitude', 'origem_longitude', 'dest_realizado_latitude', 'dest_realizado_longitude', 'distancia']],
              on=['origem_latitude', 'origem_longitude', 'dest_realizado_latitude', 'dest_realizado_longitude'],
              how='left')

In [45]:
# Cria coluna com tempo em horas a partir do campo em segundos fornecido pela Google API
#df.insert(7, 'duracao_min_Google', df['duracao_seg_Google'] / 60)
df_d['duracao_horas_Google'] = df_d['duracao_value_Google'] / 60 /60

# Merge da variavel duracaoGoogle para o par origem-destino
df = df.merge(df_d[['origem_latitude', 'origem_longitude', 'dest_realizado_latitude', 'dest_realizado_longitude', 'duracao_horas_Google']],
              on=['origem_latitude', 'origem_longitude', 'dest_realizado_latitude', 'dest_realizado_longitude'],
              how='left')

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113580 entries, 0 to 113579
Data columns (total 16 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   dt_inicio                 113580 non-null  datetime64[ns]
 1   dt_conclusao              113580 non-null  datetime64[ns]
 2   peso_bruto                113580 non-null  float64       
 3   origem_latitude           113580 non-null  float64       
 4   origem_longitude          113580 non-null  float64       
 5   dest_realizado_latitude   113580 non-null  float64       
 6   dest_realizado_longitude  113580 non-null  float64       
 7   orig&dest                 113580 non-null  object        
 8   HDD                       113580 non-null  category      
 9   DDS                       113580 non-null  object        
 10  duracao                   113580 non-null  float64       
 11  elevacao_origem           113580 non-null  float64       
 12  el

In [47]:
# Verifica as linhas com NaN
df1 = df[df.isna().any(axis=1)]
df1.head()

,dt_inicio,dt_conclusao,peso_bruto,origem_latitude,origem_longitude,dest_realizado_latitude,dest_realizado_longitude,orig&dest,HDD,DDS,duracao,elevacao_origem,elevacao_destino,delta_elevacao,distancia,duracao_horas_Google
212,2019-04-17 11:36:56,2019-04-17 13:29:24,221.58,-23.262304,-45.950952,-23.262304,-45.950952,812000 812000,11,Wednesday,1.874444,573.214661,573.214661,0.0,NaN,NaN
401,2019-05-24 13:35:23,2019-05-28 10:34:30,36.80,-26.950575,-48.750261,-26.950575,-48.750261,927800 927800,13,Friday,92.985278,13.240047,13.240047,0.0,NaN,NaN
463,2020-06-15 20:11:53,2020-06-18 08:35:20,136.00,-29.989934,-51.176718,-29.989934,-51.176718,1017801 817600,20,Monday,60.390833,4.021640,4.021640,0.0,NaN,NaN
545,2020-12-01 18:54:16,2020-12-02 11:02:15,3000.00,-23.681718,-46.697327,-23.681718,-46.697327,817900 817900,18,Tuesday,16.133056,726.457642,726.457642,0.0,NaN,NaN
609,2021-09-01 13:39:47,2021-09-02 11:38:44,2130.00,-19.947702,-44.160691,-19.947702,-44.160691,617700 617700,13,Wednesday,21.982500,847.584045,847.584045,0.0,NaN,NaN


In [48]:
# Conta as linhas com NaN
len(df1)

682

In [49]:
# Exclui linhas com NaN na coluna 'distancia'
df.dropna(subset=['distancia'], inplace=True)

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112898 entries, 0 to 113579
Data columns (total 16 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   dt_inicio                 112898 non-null  datetime64[ns]
 1   dt_conclusao              112898 non-null  datetime64[ns]
 2   peso_bruto                112898 non-null  float64       
 3   origem_latitude           112898 non-null  float64       
 4   origem_longitude          112898 non-null  float64       
 5   dest_realizado_latitude   112898 non-null  float64       
 6   dest_realizado_longitude  112898 non-null  float64       
 7   orig&dest                 112898 non-null  object        
 8   HDD                       112898 non-null  category      
 9   DDS                       112898 non-null  object        
 10  duracao                   112898 non-null  float64       
 11  elevacao_origem           112898 non-null  float64       
 12  el

## 5. Exportação do Dataset

In [52]:
# Salva dataframe com o dataset tratado
df.to_pickle(output_file)